In [3]:
import scipy

data_path = "/home/yichen/repo/cfd/myFNO/data/zongyi/NavierStokes_V1e-5_N1200_T20.mat"
data = scipy.io.loadmat(data_path)

In [8]:
import numpy as np
np.sum(data['u'][0, :, :, 0] - data['a'][0, :, :])

0.08791685

In [ ]:
import types


In [1]:
from neuralop.datasets.autoregressive_dataset import load_autoregressive_traintestsplit
data_path = "/home/yichen/repo/cfd/myFNO/data/zongyi/NavierStokes_V1e-5_N1200_T20.mat"

n_train = 1000
n_test = 200
batch_size = 32
test_batch_size = 64
train_subsample_rate = 1
test_subsample_rate = 1
time_step = 1
train_loader, test_loader = load_autoregressive_traintestsplit(
    data_path,
    n_train, n_test,
    batch_size, test_batch_size, 
    train_subsample_rate, test_subsample_rate,
    time_step,
    predict_feature='u',
)

In [2]:
import torch
import matplotlib.pyplot as plt
import sys
from neuralop.models import TFNO
from neuralop import Trainer
from torch.utils.data import Dataset, DataLoader

from neuralop.datasets.autoregressive_dataset import AutoregressiveDataset
from torch.utils.data import DataLoader
import h5py
import numpy as np

In [5]:
from torch.utils.data import DataLoader
data_path = "./neuralop/datasets/data/ns_random_forces_toy1.h5"
data = h5py.File(data_path, 'r')
print(type(data['f']),type(data))

dataset1 = AutoregressiveDataset(data, subsample_rate=4, time_step=8)
dataloader1 = DataLoader(dataset1, batch_size=2, shuffle=False)

for i, data in enumerate(dataloader1):
    for name in data:
        print(name, data[name].shape)
    break

<class 'h5py._hl.dataset.Dataset'> <class 'h5py._hl.files.File'>
y torch.Size([2, 64, 64])
f torch.Size([2, 64, 64])
mu torch.Size([2])
x torch.Size([2, 64, 64])


In [1]:
# First alter the old dataset to fit the neuraloperator scheme:
# The old dataset is from F-FNO (Tran. etl)
import h5py
import numpy as np

data_path = "./neuralop/datasets/data/ns_random_forces_.h5"
data_f = h5py.File(data_path, 'r')

print("Objects in the HDF5 file:")
for name in data_f:
    print(name)
    for dataset_name in data_f[name]:
        dataset = data_f[name][dataset_name]
        print(dataset_name, dataset.shape)

Objects in the HDF5 file:
a_temp


TypeError: Only 1D arrays allowed for fancy indexing

In [1]:
import h5py
import numpy as np

def merge_dataset(f_in, f_out):
    # Copy 'train' datasets if not exist in output
    for data_name in f_in['train']:
        if data_name not in f_out:
            f_out.create_dataset(data_name, data=f_in['train'][data_name])

    for group_name in ['test', 'valid']:
        for data_name in f_in[group_name]:
            data = f_in[group_name][data_name][:]
            if data_name in f_out:
                # Concatenate and replace existing dataset
                existing_data = f_out[data_name][:]
                concatenated_data = np.concatenate((existing_data, data), axis=0)
                del f_out[data_name]  # Delete the old dataset
                f_out.create_dataset(data_name, data=concatenated_data)  # Create new with concatenated data
            else:
                # Create new dataset if not exist
                f_out.create_dataset(data_name, data=data)

input_file = "./neuralop/datasets/data/ns_random_forces_mini.h5"
output_file = "./neuralop/datasets/data/ns_random_forces_1.h5"
with h5py.File(input_file, 'r') as f_in:
    with h5py.File(output_file, 'a') as f_out:  # Open in append mode
        merge_dataset(f_in, f_out)

        # Read the data into memory
        a_data = f_out['a'][:]
        u_data = f_out['u'][:]

        # Process the data
        expanded_a_data = np.expand_dims(a_data, axis=-1)
        concatenated_data = np.concatenate((expanded_a_data, u_data), axis=-1)

        # Replace the 'u' dataset
        del f_out['u']  # Delete the existing 'u' dataset
        f_out.create_dataset('u', data=concatenated_data)

        # Optionally, delete the 'a' dataset if no longer needed
        del f_out['a']

: 

In [4]:
class test_class:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
    
class test_subclass(test_class):
    def __init__(self, name, **kargs):
        super().__init__(name)
        self.name = name + "!"
        self.kargs = kargs
        self.age = kargs['age']

    def __str__(self):
        return self.name

a=1


In [1]:
import h5py

def copy_h5_contents(old_filename, new_filename, include_groups=None):
    """
    Copy specific groups and datasets from an old HDF5 file to a new one.

    :param old_filename: Path to the source HDF5 file.
    :param new_filename: Path to the destination HDF5 file.
    :param include_groups: List of groups to include in the copy. If None, all groups will be copied.
    """
    with h5py.File(old_filename, 'r') as old_file, h5py.File(new_filename, 'w') as new_file:
        # If no specific groups are specified, copy everything
        if include_groups is None:
            include_groups = list(old_file.keys())

        # Copy only the specified groups
        for group in include_groups:
            if group in old_file:
                old_file.copy(group, new_file)

input_file = "./neuralop/datasets/data/ns_random_forces_1.h5"
output_file = "./neuralop/datasets/data/ns_random_forces_2.h5"
copy_h5_contents(input_file, output_file)

In [3]:
input_file = "./neuralop/datasets/data/ns_random_forces_2.h5"
output_file = "./neuralop/datasets/data/ns_random_forces_1.h5"
copy_h5_contents(input_file, output_file, include_groups=['u', 'f', 'mu'])

In [4]:
data_path = "./neuralop/datasets/data/ns_random_forces_1.h5"
data_f = h5py.File(data_path, 'r')

print("Objects in the HDF5 file:")

for data_name in data_f:
        data = data_f[data_name]
        print(data_name, data.shape)

Objects in the HDF5 file:
f (200, 256, 256)
mu (200,)
u (200, 256, 256, 201)


In [ ]:
import torch
import matplotlib.pyplot as plt
import sys
from neuralop.models import TFNO
from neuralop import Trainer
from neuralop.training import OutputEncoderCallback
from neuralop.datasets import load_darcy_flow_small
from neuralop.utils import count_params
from neuralop import LpLoss, H1Loss

device = 'cpu'

from neuralop.datasets import load_ns_contextual_h5

data_path = "./neuralop/datasets/data/ns_random_forces_toy.h5"
ssr = 4
k = 8
batch_size = 256

# train_loader, test_loaders, output_encoder = \
#     load_ns_contextual_h5(data_path, batch_size=batch_size, test_batch_sizes=[batch_size],
#                           train_resolution=64, test_resolutions=[64])

# First load the dataset
